# Mining testimonial fragments of the Holocaust

**Experience domain:**

### Load the necessary libraries

In [1]:
import sys; sys.path.insert(0, '..')
import itertools

In [2]:
import get_topic_model_concordance as topic_concordancer
from utils import blacklab, db, text
mongo = db.get_db()

In [3]:
%config Completer.use_jedi = False
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import random

### Helper functions

In [4]:
def create_contextual_query(lemmas,context_length=50):
    permutations = itertools.permutations(lemmas,len(lemmas))
    final_result = []
    for element in list(permutations):
        temp_result = []
        for el in element:
            temp_result.append('[lemma="'+el+'"]')
        temp_result = '('+('[]{0,'+str(context_length)+'}').join(temp_result)+')'
        final_result.append(temp_result)
    final_result = '|'.join(final_result)
    return final_result
        
        
            

In [5]:
from utils import blacklab, db, text
import requests
import json
def find_sentence_id(label):
    props = {'annotators': 'tokenize'}

    # set the encoding of the annotator
    requests.encoding = 'utf-8'
    # make a request
    r = requests.post('http://localhost:9000/', params={'properties':
                      json.dumps(props)},
                      data=label.encode('utf-8'))
    result = json.loads(r.text, encoding='utf-8')
    query = []
    for i, token in enumerate(result['tokens']):

        if ('...'in token['word'] and ((i == 0) or
           i == len(result['tokens']) - 1)):
            continue
        elif ('...'in token['word']):
            query.append('[]{0,50}')
        elif ('-'in token['word']):
            query.append('[]{0,3}')
        elif ("n't"in token['word']):
            query.append('[]')
        elif ("'re"in token['word']):
            query.append('[]')
        elif ("?"in token['word']):
            query.append('[]')
        elif ("."in token['word']):
            query.append('[]')
        elif ("'s"in token['word']):
            query.append('[]')
        elif (","in token['word']):
            query.append('[]')
        else:
            query.append('["' + token['word'] + '"]')

    query = ' '.join(query)
    try:
        sentence = blacklab.search_blacklab(query, window=0,
                                            lemma=False,
                                            include_match=True)
        token_end = sentence[0]['token_end']
        token_start = sentence[0]['token_start']
        print (sentence[0])
        mongo = db.get_db()
        results = mongo.tokens.find({'testimony_id':
                                    sentence[0]['testimony_id']},
                                    {'_id': 0})
        tokens = list(results)[0]['tokens']
        sentenceStart = tokens[token_start]['sentence_index']
        sentenceEnd = tokens[token_end]['sentence_index']
        originalsentence = sentence[0]['complete_match']
        return (sentenceStart,sentenceEnd,sentence[0]['testimony_id'])
    except:
        print("The following query returned a null result")
        print(query)
        
            


In [6]:
def create_parent_node(label):
    """Generate a root node for a tree structure."""
    testimony_id = random.randint(1, 20)
    node = {}
    node['label'] = label
    fragment = {'label': label,
                'essay_id': random.randint(1, 20),
                'tree': get_node(testimony_id, node, is_parent=True)}
    fragment['tree']['label'] = label

    return fragment

In [7]:
def get_node(testimony_id, node, is_parent=False):
    """Generate a parent or leaf node for a tree structure."""
    if is_parent:
        return {
            'label': node['label'],
            'testimony_id': random.randint(1, 20),
            'media_index': random.randint(1, 20),
            'media_offset': random.randint(1, 20),
            'start_sentence_index': random.randint(1, 20),
            'end_sentence_index': random.randint(1, 20),
            'children': [], }
    else:
        return {'label': node['label'],
                'testimony_id': node['testimony_id'],
                'media_index': float(node['media_index']),
                'media_offset': float(node['media_offset']),
                'start_sentence_index': float(node['start_sentence_index']),
                'end_sentence_index': float(node['end_sentence_index']),
                'children': [], }

In [8]:
def check_if_main_node_exist(node):
    results = mongo.fragments.find({'label':node},{'_id': 0})
    if len(results[0])==0:
        return False
    else:
        return True

In [9]:
def add_main_node(label):
    mongo.fragments.insert(create_parent_node(label))

In [10]:
def delete_main_node(label):
    mongo.fragments.delete_one({'label':label})

In [11]:
def add_testimonial_fragments(fragments):
    if check_if_main_node_exist(fragments['main_node']):
        results = mongo.fragments.find({'label':fragments['main_node']},{'_id':0})[0]
        mid_nodes = [element['label'] for element in results['tree']['children']]
        if fragments['mid_node'] in mid_nodes:
            print ("mid node exists cannot be added")
        else:
            
            mid_node = get_node('r',{'label':fragments['mid_node']},is_parent=True)
            for fragment in fragments['fragments']:
                leaf = get_node(fragment['testimony_id'],fragment)
                mid_node['children'].append(leaf)
            results['tree']['children'].append(mid_node)
            mongo.fragments.replace_one({'label':fragments['main_node']},results)

### Add the main node

In [12]:
main_node = "sleep"
delete_main_node(main_node)
add_main_node(main_node)

/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


### Set up the query

In [13]:
query = '[lemma="sleep"]'

result = topic_concordancer.main(query,window=25,topicn=25)

### Print the key topics

for i,element in enumerate(result['topic_documents']):
    print (i)
    topic_words =  element['topic_words'][1]
    print (topic_words)
    print ('\n')

### Analyze documents

i=1
for text in result['topic_documents'][i]['texts'][0:25]:
    print (text['matched_text_words'])
    print ('\n')

## Testimonial fragments

### 1.  Cold

In [14]:
lemmas = ["cold","sleep"]

In [15]:
query = create_contextual_query(lemmas,context_length=25)
print (query)

([lemma="cold"][]{0,25}[lemma="sleep"])|([lemma="sleep"][]{0,25}[lemma="cold"])


In [16]:
domain_term = "cold"

In [17]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [18]:
fragment_1 = {}
fragment_1['original_sentence'] = "And we went to sleep. But in the winter time, it was very cold. "
fragment_1['label']="And we went to sleep. But in the winter time, it was very cold. "
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22we%22%5D+%5B%22went%22%5D+%5B%22to%22%5D+%5B%22sleep%22%5D+%5B%5D+%5B%22But%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22winter%22%5D+%5B%22time%22%5D+%5B%5D+%5B%22it%22%5D+%5B%22was%22%5D+%5B%22very%22%5D+%5B%22cold%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And we went to sleep . But in the winter time , it was very cold . ', 'right': '', 'complete_match': 'And we went to sleep . But in the winter time , it was very cold . ', 'testimony_id': 'HVT-70', 'shelfmark': ['Fortunoff Archive HVT-70'], 'token_start': 4564, 'token_end': 4581}


In [19]:
fragment_2 = {}
fragment_2['original_sentence'] = "It was dark, cold, wet. I couldn't sleep."
fragment_2['label']="It was dark, cold, wet. I couldn't sleep."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22It%22%5D+%5B%22was%22%5D+%5B%22dark%22%5D+%5B%5D+%5B%22cold%22%5D+%5B%5D+%5B%22wet%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22could%22%5D+%5B%5D+%5B%22sleep%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "It was dark , cold , wet . I could n't sleep . ", 'right': '', 'complete_match': "It was dark , cold , wet . I could n't sleep . ", 'testimony_id': 'irn504467', 'shelfmark': ['USHMM RG-50.030*0001'], 'token_start': 12007, 'token_end': 12020}


In [20]:
fragment_3 = {}
fragment_3['original_sentence'] = "We were sleeping outside the houses, on the steps "
fragment_3['label']="We were sleeping outside the houses, on the steps outside-- in the cold, in the fields."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22We%22%5D+%5B%22were%22%5D+%5B%22sleeping%22%5D+%5B%22outside%22%5D+%5B%22the%22%5D+%5B%22houses%22%5D+%5B%5D+%5B%22on%22%5D+%5B%22the%22%5D+%5B%22steps%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'We were sleeping outside the houses , on the steps ', 'right': '', 'complete_match': 'We were sleeping outside the houses , on the steps ', 'testimony_id': 'HVT-145', 'shelfmark': ['Fortunoff Archive HVT-145'], 'token_start': 6736, 'token_end': 6746}


In [21]:
fragment_4 = {}
fragment_4['original_sentence'] = "I fact, I was never so cold in my life as I was in May of 1942. I slept on the upper bunk and the wind came in from the open rafters under the roof, and we had no bedding or blankets and slept in our day clothing, which consisted of a shirt and a cotton blouse of sorts."
fragment_4['label']= "I was never so cold in my life (..)I slept on the upper bunk and the wind came in from the open rafters(..)"
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22fact%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22never%22%5D+%5B%22so%22%5D+%5B%22cold%22%5D+%5B%22in%22%5D+%5B%22my%22%5D+%5B%22life%22%5D+%5B%22as%22%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22in%22%5D+%5B%22May%22%5D+%5B%22of%22%5D+%5B%221942%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22slept%22%5D+%5B%22on%22%5D+%5B%22the%22%5D+%5B%22upper%22%5D+%5B%22bunk%22%5D+%5B%22and%22%5D+%5B%22the%22%5D+%5B%22wind%22%5D+%5B%22came%22%5D+%5B%22in%22%5D+%5B%22from%22%5D+%5B%22the%22%5D+%5B%22open%22%5D+%5B%22rafters%22%5D+%5B%22under%22%5D+%5B%22the%22%5D+%5B%22roof%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22we%22%5D+%5B%22had%22%5D+%5B%22no%22%5D+%5B%22bedding%22%5D+%5B%22or%22%5D+%5B%22blankets%22%5D+%5B%22and%22%5D+%5B%22slept%22%5D+%5B%22in%22%5D+%5B%22our%22%5D+%5B%22day%22%5D+%5B%22clothing%22%5D+%5B%5D+%5B%22which%22%5D+%5B%22consisted%22%5D+%5B%22of%22%5D+%5B%22a%22%5D+%5B%22shirt%22%5D+%5B%22and%22%5D+%5B%22a%22%5D+%5B%22cotto

In [22]:
fragment_5 = {}
fragment_5['original_sentence'] = "Of course, we couldn’t sleep for it was too cold."
fragment_5['label']= "Of course, we couldn’t sleep for it was too cold."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Of%22%5D+%5B%22course%22%5D+%5B%5D+%5B%22we%22%5D+%5B%22could%22%5D+%5B%5D+%5B%22sleep%22%5D+%5B%22for%22%5D+%5B%22it%22%5D+%5B%22was%22%5D+%5B%22too%22%5D+%5B%22cold%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'Of course , we could n’t sleep for it was too cold . ', 'right': '', 'complete_match': 'Of course , we could n’t sleep for it was too cold . ', 'testimony_id': 'irn509119', 'shelfmark': ['USHMM RG-50.233*0037'], 'token_start': 9608, 'token_end': 9621}


In [23]:
add_testimonial_fragments(fragments)

### 2.  

In [24]:
lemmas = ["dead","sleep"]

In [25]:
query = create_contextual_query(lemmas,context_length=25)
print (query)

([lemma="dead"][]{0,25}[lemma="sleep"])|([lemma="sleep"][]{0,25}[lemma="dead"])


In [26]:
domain_term = "with a dead"

In [27]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [28]:
fragment_1 = {}
fragment_1['original_sentence'] = "And I slept with a dead woman because she was very nice and cool."
fragment_1['label']="And I slept with a dead woman because she was very nice and cool."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22I%22%5D+%5B%22slept%22%5D+%5B%22with%22%5D+%5B%22a%22%5D+%5B%22dead%22%5D+%5B%22woman%22%5D+%5B%22because%22%5D+%5B%22she%22%5D+%5B%22was%22%5D+%5B%22very%22%5D+%5B%22nice%22%5D+%5B%22and%22%5D+%5B%22cool%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And I slept with a dead woman because she was very nice and cool . ', 'right': '', 'complete_match': 'And I slept with a dead woman because she was very nice and cool . ', 'testimony_id': 'HVT-77', 'shelfmark': ['Fortunoff Archive HVT-77'], 'token_start': 4258, 'token_end': 4273}


In [29]:
fragment_2 = {}
fragment_2['original_sentence'] = "I just slept there the rest of the night, but he was dead just from cold and not eating."
fragment_2['label']="I just slept there the rest of the night, but he was dead just from cold and not eating."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22just%22%5D+%5B%22slept%22%5D+%5B%22there%22%5D+%5B%22the%22%5D+%5B%22rest%22%5D+%5B%22of%22%5D+%5B%22the%22%5D+%5B%22night%22%5D+%5B%5D+%5B%22but%22%5D+%5B%22he%22%5D+%5B%22was%22%5D+%5B%22dead%22%5D+%5B%22just%22%5D+%5B%22from%22%5D+%5B%22cold%22%5D+%5B%22and%22%5D+%5B%22not%22%5D+%5B%22eating%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'I just slept there the rest of the night , but he was dead just from cold and not eating . ', 'right': '', 'complete_match': 'I just slept there the rest of the night , but he was dead just from cold and not eating . ', 'testimony_id': 'irn504694', 'shelfmark': ['USHMM RG-50.030*0200'], 'token_start': 13396, 'token_end': 13417}


In [30]:
fragment_4 = {}
fragment_4['original_sentence'] = "you sleep with the dead"
fragment_4['label']= "(..)you sleep with the dead(..)"
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22you%22%5D+%5B%22sleep%22%5D+%5B%22with%22%5D+%5B%22the%22%5D+%5B%22dead%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'you sleep with the dead ', 'right': '', 'complete_match': 'you sleep with the dead ', 'testimony_id': 'irn508628', 'shelfmark': ['USHMM RG-50.462*0007'], 'token_start': 19960, 'token_end': 19965}


In [31]:
fragment_5 = {}
fragment_5['original_sentence'] = "when you got up in the morning, you thought you sleeping with somebody and in the meantime the other guy was already dead."
fragment_5['label']= "(..) when you got up in the morning, you thought you sleeping with somebody and in the meantime the other guy was already dead."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22when%22%5D+%5B%22you%22%5D+%5B%22got%22%5D+%5B%22up%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22morning%22%5D+%5B%5D+%5B%22you%22%5D+%5B%22thought%22%5D+%5B%22you%22%5D+%5B%22sleeping%22%5D+%5B%22with%22%5D+%5B%22somebody%22%5D+%5B%22and%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22meantime%22%5D+%5B%22the%22%5D+%5B%22other%22%5D+%5B%22guy%22%5D+%5B%22was%22%5D+%5B%22already%22%5D+%5B%22dead%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'when you got up in the morning , you thought you sleeping with somebody and in the meantime the other guy was already dead . ', 'right': '', 'complete_match': 'when you got up in the morning , you thought you sleeping with somebody and in the meantime the other guy was already dead . ', 'testimony_id': 'irn509081', 'shelfmark': ['USHMM RG-50.233*0004'], 'token_start': 4330, 'token_end': 4355}


In [32]:
add_testimonial_fragments(fragments)

### 3.  

In [33]:
lemmas = ["n't","sleep"]

In [34]:
query = create_contextual_query(lemmas,context_length=1)
print (query)

([lemma="n't"][]{0,1}[lemma="sleep"])|([lemma="sleep"][]{0,1}[lemma="n't"])


In [35]:
domain_term = "could not"

In [36]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [37]:
fragment_1 = {}
fragment_1['original_sentence'] = "not sleep. We couldn't."
fragment_1['label']="We couldn't even-- not sleep. We couldn't."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22not%22%5D+%5B%22sleep%22%5D+%5B%5D+%5B%22We%22%5D+%5B%22could%22%5D+%5B%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "not sleep . We could n't . ", 'right': '', 'complete_match': "not sleep . We could n't . ", 'testimony_id': 'usc_shoah_10272', 'shelfmark': ['USC Shoah Foundation 10272'], 'token_start': 21372, 'token_end': 21379}


In [38]:
fragment_2 = {}
fragment_2['original_sentence'] = "Well, we had enough time, but who could sleep? "
fragment_2['label']="Well, we had enough time, but who could sleep? "
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Well%22%5D+%5B%5D+%5B%22we%22%5D+%5B%22had%22%5D+%5B%22enough%22%5D+%5B%22time%22%5D+%5B%5D+%5B%22but%22%5D+%5B%22who%22%5D+%5B%22could%22%5D+%5B%22sleep%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'Well , we had enough time , but who could sleep ? ', 'right': '', 'complete_match': 'Well , we had enough time , but who could sleep ? ', 'testimony_id': 'usc_shoah_20745', 'shelfmark': ['USC Shoah Foundation 20745'], 'token_start': 10578, 'token_end': 10590}


In [39]:
fragment_3 = {}
fragment_3['original_sentence'] = "And no sleep and no food. "
fragment_3['label']="And no sleep and no food. "
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22no%22%5D+%5B%22sleep%22%5D+%5B%22and%22%5D+%5B%22no%22%5D+%5B%22food%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And no sleep and no food . ', 'right': '', 'complete_match': 'And no sleep and no food . ', 'testimony_id': 'usc_shoah_11184', 'shelfmark': ['USC Shoah Foundation 11184'], 'token_start': 14709, 'token_end': 14716}


In [40]:
fragment_4 = {}
fragment_4['original_sentence'] = "And it's snowing, no food, no sleep. "
fragment_4['label']= "And it's snowing, no food, no sleep. "
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22it%22%5D+%5B%5D+%5B%22snowing%22%5D+%5B%5D+%5B%22no%22%5D+%5B%22food%22%5D+%5B%5D+%5B%22no%22%5D+%5B%22sleep%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "And it 's snowing , no food , no sleep . ", 'right': '', 'complete_match': "And it 's snowing , no food , no sleep . ", 'testimony_id': 'usc_shoah_895', 'shelfmark': ['USC Shoah Foundation 895'], 'token_start': 7284, 'token_end': 7295}


In [41]:
fragment_5 = {}
fragment_5['original_sentence'] = "There was no sleep, not much sleep at all."
fragment_5['label']= "There was no sleep, not much sleep at all."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22There%22%5D+%5B%22was%22%5D+%5B%22no%22%5D+%5B%22sleep%22%5D+%5B%5D+%5B%22not%22%5D+%5B%22much%22%5D+%5B%22sleep%22%5D+%5B%22at%22%5D+%5B%22all%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'There was no sleep , not much sleep at all . ', 'right': '', 'complete_match': 'There was no sleep , not much sleep at all . ', 'testimony_id': 'usc_shoah_11184', 'shelfmark': ['USC Shoah Foundation 11184'], 'token_start': 14829, 'token_end': 14840}


In [42]:
add_testimonial_fragments(fragments)

### 4.  

In [43]:
lemmas = ["cry","sleep"]

In [44]:
query = create_contextual_query(lemmas,context_length=25)
print (query)

([lemma="cry"][]{0,25}[lemma="sleep"])|([lemma="sleep"][]{0,25}[lemma="cry"])


In [45]:
domain_term = "cry to sleep"

In [46]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [47]:
fragment_1 = {}
fragment_1['original_sentence'] = "And I cried myself to sleep because of the hard work. And over there, that's the way I was for six months."
fragment_1['label']="And I cried myself to sleep because of the hard work. (..) that's the way I was for six months."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22I%22%5D+%5B%22cried%22%5D+%5B%22myself%22%5D+%5B%22to%22%5D+%5B%22sleep%22%5D+%5B%22because%22%5D+%5B%22of%22%5D+%5B%22the%22%5D+%5B%22hard%22%5D+%5B%22work%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22over%22%5D+%5B%22there%22%5D+%5B%5D+%5B%22that%22%5D+%5B%5D+%5B%22the%22%5D+%5B%22way%22%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22for%22%5D+%5B%22six%22%5D+%5B%22months%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "And I cried myself to sleep because of the hard work . And over there , that 's the way I was for six months . ", 'right': '', 'complete_match': "And I cried myself to sleep because of the hard work . And over there , that 's the way I was for six months . ", 'testimony_id': 'irn504640', 'shelfmark': ['USHMM RG-50.030*0146'], 'token_start': 6869, 'token_end': 6895}


In [48]:
fragment_2 = {}
fragment_2['original_sentence'] = "We had to carry them out uh 2 by 2, and it was very, very heavy, and the labor was just as hard as those tremendous rocks that we were dragging, and I, I thought I was going to do, I, I cried myself to sleep every night because I thought I couldn't do that hard labor."
fragment_2['label']="I cried myself to sleep every night because I thought I couldn't do that hard labor."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22We%22%5D+%5B%22had%22%5D+%5B%22to%22%5D+%5B%22carry%22%5D+%5B%22them%22%5D+%5B%22out%22%5D+%5B%22uh%22%5D+%5B%222%22%5D+%5B%22by%22%5D+%5B%222%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22it%22%5D+%5B%22was%22%5D+%5B%22very%22%5D+%5B%5D+%5B%22very%22%5D+%5B%22heavy%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22the%22%5D+%5B%22labor%22%5D+%5B%22was%22%5D+%5B%22just%22%5D+%5B%22as%22%5D+%5B%22hard%22%5D+%5B%22as%22%5D+%5B%22those%22%5D+%5B%22tremendous%22%5D+%5B%22rocks%22%5D+%5B%22that%22%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22dragging%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22I%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22thought%22%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22going%22%5D+%5B%22to%22%5D+%5B%22do%22%5D+%5B%5D+%5B%22I%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22cried%22%5D+%5B%22myself%22%5D+%5B%22to%22%5D+%5B%22sleep%22%5D+%5B%22every%22%5D+%5B%22night%22%5D+%5B%22because%22%5D+%5B%22I%22%5D+%5B%22thought%22%5D+%5B%22I%22%5D+%5B%22could%22%5D+%5B%5D+%5B%22do%22

In [49]:
fragment_3 = {}
fragment_3['original_sentence'] = " I just cried myself to sleep many nights"
fragment_3['label']="I just cried myself to sleep many nights"
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22just%22%5D+%5B%22cried%22%5D+%5B%22myself%22%5D+%5B%22to%22%5D+%5B%22sleep%22%5D+%5B%22many%22%5D+%5B%22nights%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'I just cried myself to sleep many nights ', 'right': '', 'complete_match': 'I just cried myself to sleep many nights ', 'testimony_id': 'irn84819', 'shelfmark': ['USHMM RG-50.030*0754'], 'token_start': 2445, 'token_end': 2453}


In [50]:
fragment_4 = {}
fragment_4['original_sentence'] = "And as I cried, I cried myself to sleep, I dozed off, and I saw my mother coming in through the camp"
fragment_4['label']= "And as I cried, I cried myself to sleep, I dozed off, and I saw my mother coming in through the camp (..)"
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22as%22%5D+%5B%22I%22%5D+%5B%22cried%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22cried%22%5D+%5B%22myself%22%5D+%5B%22to%22%5D+%5B%22sleep%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22dozed%22%5D+%5B%22off%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22I%22%5D+%5B%22saw%22%5D+%5B%22my%22%5D+%5B%22mother%22%5D+%5B%22coming%22%5D+%5B%22in%22%5D+%5B%22through%22%5D+%5B%22the%22%5D+%5B%22camp%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And as I cried , I cried myself to sleep , I dozed off , and I saw my mother coming in through the camp ', 'right': '', 'complete_match': 'And as I cried , I cried myself to sleep , I dozed off , and I saw my mother coming in through the camp ', 'testimony_id': 'irn505576', 'shelfmark': ['USHMM RG-50.042*0023'], 'token_start': 8342, 'token_end': 8367}


In [51]:
add_testimonial_fragments(fragments)

### 5.  

In [52]:
lemmas = ["straw","sleep"]

In [53]:
query = create_contextual_query(lemmas,context_length=25)
print (query)

([lemma="straw"][]{0,25}[lemma="sleep"])|([lemma="sleep"][]{0,25}[lemma="straw"])


In [54]:
domain_term = "straw"

In [55]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [56]:
fragment_1 = {}
fragment_1['original_sentence'] = "And we slept on wooden planks with filthy straw or something."
fragment_1['label']="And we slept on wooden planks with filthy straw or something."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22we%22%5D+%5B%22slept%22%5D+%5B%22on%22%5D+%5B%22wooden%22%5D+%5B%22planks%22%5D+%5B%22with%22%5D+%5B%22filthy%22%5D+%5B%22straw%22%5D+%5B%22or%22%5D+%5B%22something%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And we slept on wooden planks with filthy straw or something . ', 'right': '', 'complete_match': 'And we slept on wooden planks with filthy straw or something . ', 'testimony_id': 'irn504441', 'shelfmark': ['USHMM RG-50.106*0019'], 'token_start': 2924, 'token_end': 2936}


In [57]:
fragment_2 = {}
fragment_2['original_sentence'] = "And we slept on some straw mattresses."
fragment_2['label']="And we slept on some straw mattresses."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22we%22%5D+%5B%22slept%22%5D+%5B%22on%22%5D+%5B%22some%22%5D+%5B%22straw%22%5D+%5B%22mattresses%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And we slept on some straw mattresses . ', 'right': '', 'complete_match': 'And we slept on some straw mattresses . ', 'testimony_id': 'HVT-83', 'shelfmark': ['Fortunoff Archive HVT-83'], 'token_start': 7458, 'token_end': 7466}


In [58]:
fragment_3 = {}
fragment_3['original_sentence'] = "We slept on straw. In the barracks there were 50-60 other men."
fragment_3['label']="We slept on straw. In the barracks there were 50-60 other men."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22We%22%5D+%5B%22slept%22%5D+%5B%22on%22%5D+%5B%22straw%22%5D+%5B%5D+%5B%22In%22%5D+%5B%22the%22%5D+%5B%22barracks%22%5D+%5B%22there%22%5D+%5B%22were%22%5D+%5B%5D%7B0%2C3%7D+%5B%22other%22%5D+%5B%22men%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'We slept on straw . In the barracks there were 50-60 other men . ', 'right': '', 'complete_match': 'We slept on straw . In the barracks there were 50-60 other men . ', 'testimony_id': 'irn507461', 'shelfmark': ['USHMM RG-50.031*0029'], 'token_start': 1110, 'token_end': 1124}


In [59]:
fragment_4 = {}
fragment_4['original_sentence'] = "The beds, the bunks, yes. And it was straw there. That's all."
fragment_4['label']= "The beds, the bunks, yes. And it was straw there. That's all."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22The%22%5D+%5B%22beds%22%5D+%5B%5D+%5B%22the%22%5D+%5B%22bunks%22%5D+%5B%5D+%5B%22yes%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22it%22%5D+%5B%22was%22%5D+%5B%22straw%22%5D+%5B%22there%22%5D+%5B%5D+%5B%22That%22%5D+%5B%5D+%5B%22all%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "The beds , the bunks , yes . And it was straw there . That 's all . ", 'right': '', 'complete_match': "The beds , the bunks , yes . And it was straw there . That 's all . ", 'testimony_id': 'usc_shoah_16922', 'shelfmark': ['USC Shoah Foundation 16922'], 'token_start': 7651, 'token_end': 7669}


In [60]:
fragment_5 = {}
fragment_5['original_sentence'] = "And they put straw on the floor, so we all slept on straw, called paille in French."
fragment_5['label']= "And they put straw on the floor, so we all slept on straw, called paille in French."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22they%22%5D+%5B%22put%22%5D+%5B%22straw%22%5D+%5B%22on%22%5D+%5B%22the%22%5D+%5B%22floor%22%5D+%5B%5D+%5B%22so%22%5D+%5B%22we%22%5D+%5B%22all%22%5D+%5B%22slept%22%5D+%5B%22on%22%5D+%5B%22straw%22%5D+%5B%5D+%5B%22called%22%5D+%5B%22paille%22%5D+%5B%22in%22%5D+%5B%22French%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And they put straw on the floor , so we all slept on straw , called paille in French . ', 'right': '', 'complete_match': 'And they put straw on the floor , so we all slept on straw , called paille in French . ', 'testimony_id': 'usc_shoah_19923', 'shelfmark': ['USC Shoah Foundation 19923'], 'token_start': 10997, 'token_end': 11017}


In [61]:
add_testimonial_fragments(fragments)

### 6. sleep  

In [62]:
lemmas = ["afraid","sleep"]

In [63]:
query = create_contextual_query(lemmas,context_length=2)
print (query)

([lemma="afraid"][]{0,2}[lemma="sleep"])|([lemma="sleep"][]{0,2}[lemma="afraid"])


In [64]:
domain_term = "afraid"

In [65]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [66]:
fragment_1 = {}
fragment_1['original_sentence'] = "And one day, we were afraid-- there were raids all over, so we were afraid to sleep in any place."
fragment_1['label']="(..) there were raids all over, so we were afraid to sleep in any place"
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22one%22%5D+%5B%22day%22%5D+%5B%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22afraid%22%5D+%5B%5D%7B0%2C3%7D+%5B%22there%22%5D+%5B%22were%22%5D+%5B%22raids%22%5D+%5B%22all%22%5D+%5B%22over%22%5D+%5B%5D+%5B%22so%22%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22afraid%22%5D+%5B%22to%22%5D+%5B%22sleep%22%5D+%5B%22in%22%5D+%5B%22any%22%5D+%5B%22place%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And one day , we were afraid -- there were raids all over , so we were afraid to sleep in any place . ', 'right': '', 'complete_match': 'And one day , we were afraid -- there were raids all over , so we were afraid to sleep in any place . ', 'testimony_id': 'HVT-109', 'shelfmark': ['Fortunoff Archive HVT-109'], 'token_start': 7719, 'token_end': 7743}


In [67]:
fragment_2 = {}
fragment_2['original_sentence'] = "I remember I was afraid to sleep in my bed because after all...uh...the two, three weeks I didn't sleep in a bed."
fragment_2['label']="I remember I was afraid to sleep in my bed (..)"
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22remember%22%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22afraid%22%5D+%5B%22to%22%5D+%5B%22sleep%22%5D+%5B%22in%22%5D+%5B%22my%22%5D+%5B%22bed%22%5D+%5B%22because%22%5D+%5B%22after%22%5D+%5B%22all%22%5D+%5B%5D%7B0%2C50%7D+%5B%22uh%22%5D+%5B%5D%7B0%2C50%7D+%5B%22the%22%5D+%5B%22two%22%5D+%5B%5D+%5B%22three%22%5D+%5B%22weeks%22%5D+%5B%22I%22%5D+%5B%22did%22%5D+%5B%5D+%5B%22sleep%22%5D+%5B%22in%22%5D+%5B%22a%22%5D+%5B%22bed%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "I remember I was afraid to sleep in my bed because after all ... uh ... the two , three weeks I did n't sleep in a bed . ", 'right': '', 'complete_match': "I remember I was afraid to sleep in my bed because after all ... uh ... the two , three weeks I did n't sleep in a bed . ", 'testimony_id': 'irn504608', 'shelfmark': ['USHMM RG-50.030*0114'], 'token_start': 2371, 'token_end': 2400}


In [68]:
fragment_3 = {}
fragment_3['original_sentence'] = "You couldn’t sleep in the barracks, you were afraid to sleep there, they’re going to come, take you out."
fragment_3['label']="You couldn’t sleep in the barracks, you were afraid to sleep there, they’re going to come, take you out."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22You%22%5D+%5B%22could%22%5D+%5B%5D+%5B%22sleep%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22barracks%22%5D+%5B%5D+%5B%22you%22%5D+%5B%22were%22%5D+%5B%22afraid%22%5D+%5B%22to%22%5D+%5B%22sleep%22%5D+%5B%22there%22%5D+%5B%5D+%5B%22they%22%5D+%5B%5D+%5B%22going%22%5D+%5B%22to%22%5D+%5B%22come%22%5D+%5B%5D+%5B%22take%22%5D+%5B%22you%22%5D+%5B%22out%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'You could n’t sleep in the barracks , you were afraid to sleep there , they ’re going to come , take you out . ', 'right': '', 'complete_match': 'You could n’t sleep in the barracks , you were afraid to sleep there , they ’re going to come , take you out . ', 'testimony_id': 'irn510703', 'shelfmark': ['USHMM RG-50.156*0049'], 'token_start': 2382, 'token_end': 2407}


In [69]:
fragment_4 = {}
fragment_4['original_sentence'] = "I was afraid to sleep, if my dreams control me."
fragment_4['label']= "I was afraid to sleep, if my dreams control me."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22was%22%5D+%5B%22afraid%22%5D+%5B%22to%22%5D+%5B%22sleep%22%5D+%5B%5D+%5B%22if%22%5D+%5B%22my%22%5D+%5B%22dreams%22%5D+%5B%22control%22%5D+%5B%22me%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'I was afraid to sleep , if my dreams control me . ', 'right': '', 'complete_match': 'I was afraid to sleep , if my dreams control me . ', 'testimony_id': 'usc_shoah_7455', 'shelfmark': ['USC Shoah Foundation 7455'], 'token_start': 38406, 'token_end': 38418}


In [70]:
add_testimonial_fragments(fragments)